In [1]:
import os
import pandas as pd
os.chdir(r'C:\Users\신상훈\Desktop\preprocessing\part-2.-탐색적-데이터-분석\2. 탐색적 데이터 분석\데이터')

In [5]:
df['product_id'].value_counts(normalize = True).head(100)

24852    0.013524
13176    0.011180
21137    0.007868
21903    0.007066
47626    0.005875
           ...   
8193     0.001024
9387     0.000996
37687    0.000984
20995    0.000983
34243    0.000976
Name: product_id, Length: 100, dtype: float64

In [6]:
product_list_per_order = df.groupby('order_id')['product_id'].apply(list)

In [7]:
product_list_per_order

order_id
1          [49302, 11109, 10246, 49683, 43633, 13176, 472...
36         [39612, 19660, 49235, 43086, 46620, 34497, 486...
38         [11913, 18159, 4461, 21616, 23622, 32433, 2884...
96         [20574, 30391, 40706, 25610, 27966, 24489, 39275]
98         [8859, 19731, 43654, 13176, 4357, 37664, 34065...
                                 ...                        
3421049           [40800, 17706, 33424, 17299, 26800, 34243]
3421056                    [5750, 9340, 21709, 16475, 12432]
3421058    [15629, 4347, 34466, 6244, 6858, 30316, 35578,...
3421063                         [49235, 13565, 14233, 35548]
3421070                                 [35951, 16953, 4724]
Name: product_id, Length: 131209, dtype: object

In [8]:
# 구매 기록 데이터 -> One hot encoding 데이터
from mlxtend.preprocessing import TransactionEncoder
encoder = TransactionEncoder() # 인스턴스화
one_hot_df = encoder.fit(product_list_per_order).transform(product_list_per_order) # 결과 : ndarray
one_hot_df = pd.DataFrame(one_hot_df, columns = encoder.columns_)
one_hot_df.head()

,1,2,3,4,5,7,8,9,10,11,...,49677,49678,49679,49680,49681,49682,49683,49686,49687,49688
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
from mlxtend.frequent_patterns import *
frequent_item_df = apriori(one_hot_df, min_support = 0.003)
result = association_rules(frequent_item_df, metric = 'confidence', min_threshold = 0.1)
result[['antecedents', 'consequents', 'support', 'confidence']].sort_values(by = 'confidence', ascending = False).to_csv("연관규칙탐사결과.csv", index = False)

In [10]:
result.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(12022),(1815),0.014206,0.008887,0.003018,0.212446,23.906409,0.002892,1.258471
1,(1815),(12022),0.008887,0.014206,0.003018,0.339623,23.906409,0.002892,1.492773
2,(3640),(17300),0.028672,0.074568,0.004146,0.144604,1.939221,0.002008,1.081875
3,(3640),(19574),0.028672,0.142719,0.008163,0.284689,1.994754,0.004071,1.198473
4,(3640),(20648),0.028672,0.045980,0.003780,0.131845,2.867432,0.002462,1.098905


In [11]:
product_list_per_order

order_id
1          [49302, 11109, 10246, 49683, 43633, 13176, 472...
36         [39612, 19660, 49235, 43086, 46620, 34497, 486...
38         [11913, 18159, 4461, 21616, 23622, 32433, 2884...
96         [20574, 30391, 40706, 25610, 27966, 24489, 39275]
98         [8859, 19731, 43654, 13176, 4357, 37664, 34065...
                                 ...                        
3421049           [40800, 17706, 33424, 17299, 26800, 34243]
3421056                    [5750, 9340, 21709, 16475, 12432]
3421058    [15629, 4347, 34466, 6244, 6858, 30316, 35578,...
3421063                         [49235, 13565, 14233, 35548]
3421070                                 [35951, 16953, 4724]
Name: product_id, Length: 131209, dtype: object

In [1]:
import os
import numpy as np
import pandas as pd
os.chdir(r'C:\Users\신상훈\Desktop\preprocessing\part-2.-탐색적-데이터-분석\2. 탐색적 데이터 분석\데이터')

In [2]:
df = pd.read_csv("페이지내_사용자_이동.csv", engine = "python")

In [3]:
df.head()

,고객ID,방문 페이지,순서
0,0,페이지C,1
1,0,페이지E,2
2,0,페이지B,3
3,0,페이지F,4
4,0,페이지C,5


In [4]:
# 주문별 카트에 추가한 순서를 고려하기 위해, 정렬이 필요
df.sort_values(by=['고객ID','순서'], inplace = True)

In [5]:
page_set = df['방문 페이지'].unique()
page_set[:10]

array(['페이지C', '페이지E', '페이지B', '페이지F', '페이지D', '페이지J', '메인', '페이지G',
       '페이지A', '페이지I'], dtype=object)

In [6]:
page_sequence_per_order = df.groupby('고객ID')['방문 페이지'].apply(np.array)

In [7]:
page_sequence_per_order.head()

고객ID
0    [페이지C, 페이지E, 페이지B, 페이지F, 페이지C, 페이지D, 페이지J, 메인,...
1    [페이지B, 메인, 메인, 페이지A, 페이지F, 페이지C, 페이지I, 페이지E, 페...
2    [페이지F, 페이지J, 페이지D, 페이지G, 페이지C, 메인, 페이지I, 페이지J,...
3    [페이지I, 페이지I, 페이지J, 페이지J, 페이지C, 페이지A, 페이지H, 페이지...
4    [페이지J, 페이지H, 페이지G, 페이지G, 페이지E, 페이지C, 페이지A, 페이지...
Name: 방문 페이지, dtype: object